In [1]:
import polars as pl
import matplotlib.pyplot as plt
import pandas as pd
from sqlalchemy import text

from sqlalchemy import create_engine
from urllib.parse import quote_plus
import numpy as np

connection_string = (
    'mssql+pyodbc://'
    'Angel_chavez:{}@172.16.2.227\\DWHFARINTERDEV/BI_FARINTER?'
    'driver=ODBC+Driver+17+for+SQL+Server'.format(quote_plus('@ng3l_ch@v3z'))
)
engine = create_engine(connection_string)
query = text("""SELECT
    fc.Suc_Id,
    fc.Emp_Id,
    fc.Caja_Id,
    fc.Factura_Id,
    fc.Factura_Fecha,
    fc.Factura_FechaHora,
    fc.AnioMes_Id,
    fc.Cliente_Id,
    fc.MonederoTarj_Id,
    fc.Vendedor_Id,
    CASE 
        WHEN fc.Monedero_Id = '0' THEN 'No Identificada'
        ELSE md.Tipo_Plan 
    END AS Tipo_Plan,
    md.Edad,
    suc.TipoSucursal_Id,
    suc.TipoSucursal_Nombre,
    suc.Zona_Id,
    suc.Zona_Nombre
FROM
    [BI_Kielsa_Hecho_FacturaEncabezado] AS fc
    JOIN [BI_Kielsa_Dim_Monedero] AS md 
        ON md.Monedero_Id = fc.Monedero_Id AND md.Emp_Id = fc.Emp_Id       
    JOIN [BI_Kielsa_Dim_Sucursal] AS suc 
        ON suc.Sucursal_Id = fc.Suc_Id AND suc.Emp_Id = fc.Emp_Id
WHERE
    fc.AnioMes_Id IN (202410,202411,202409)
    AND fc.Emp_Id = 1;""")

with engine.connect() as conn:
    df_base = pd.read_sql(query, conn, index_col='Factura_Id')

In [2]:
df_base.head()
print(df_base.count())

Suc_Id                 2667049
Emp_Id                 2667049
Caja_Id                2667049
Factura_Fecha          2667049
Factura_FechaHora      2667049
AnioMes_Id             2667049
Cliente_Id             2667049
MonederoTarj_Id        2667049
Vendedor_Id            2667049
Tipo_Plan              2667049
Edad                   2667049
TipoSucursal_Id        2667049
TipoSucursal_Nombre    2667049
Zona_Id                2667049
Zona_Nombre            2667049
dtype: int64


In [3]:
# Crear una copia del DataFrame original
df = df_base.copy()

# Convertir los valores de hora a cadenas
df['Factura_FechaHora'] = df['Factura_FechaHora'].astype(str)

# Agregar una fecha ficticia a los valores de hora
df['Factura_FechaHora'] = '1900-01-01 ' + df['Factura_FechaHora']

# Convertir la columna Factura_FechaHora a datetime
df['Factura_FechaHora'] = pd.to_datetime(df['Factura_FechaHora'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Asegúrate de que Factura_Fecha y Factura_FechaHora sean tipo datetime
df['Factura_Fecha'] = pd.to_datetime(df['Factura_Fecha'])

# Crear el diccionario para mapear los tipos de transacciones
tipos_transaccion = {
    'Monedero tercera edad': 'Tercera Edad',
    'Monedero Cuarta Edad': 'Tercera Edad',
    'Monedero todo publico': 'Todo Publico',
    'Monedero Clínica TP': 'Todo Publico',
    'Monedero Clínica TE': 'Tercera Edad',
    'No Identificada': 'No Identificada',
}

# Mapear los tipos de transacciones y eliminar las no relevantes
df['Tipo_Plan'] = df['Tipo_Plan'].map(tipos_transaccion)
#df = df.dropna(subset=['Tipo_Plan'])

# Extraer hora y día de la semana para el agrupamiento
df['Hora'] = df['Factura_FechaHora'].dt.hour
df['Dia_Semana'] = df['Factura_Fecha'].dt.day_name()

# Agrupar por sucursal, día de la semana, hora y tipo de transacción
agrupado = df.groupby(['Suc_Id', 'Dia_Semana', 'Hora', 'Tipo_Plan','Factura_Fecha']
                      ).size().reset_index(name='Cantidad_Transacciones')

df_filter = df[(df['Suc_Id'] == 1) & (df['Dia_Semana'] == 'Monday') & (df['Hora'] == 7)]

In [4]:
print(df_filter)

            Suc_Id  Emp_Id  Caja_Id Factura_Fecha   Factura_FechaHora  \
Factura_Id                                                              
71668            1       1        2    2024-09-16 1900-01-01 07:24:39   
73884            1       1        2    2024-10-28 1900-01-01 07:38:44   
26186            1       1        1    2024-09-30 1900-01-01 07:39:38   
74573            1       1        2    2024-11-11 1900-01-01 07:40:54   
74574            1       1        2    2024-11-11 1900-01-01 07:55:15   
74234            1       1        2    2024-11-04 1900-01-01 07:19:00   
74235            1       1        2    2024-11-04 1900-01-01 07:59:37   
73883            1       1        2    2024-10-28 1900-01-01 07:38:03   
73885            1       1        2    2024-10-28 1900-01-01 07:54:09   
26158            1       1        1    2024-09-23 1900-01-01 07:23:57   
72064            1       1        2    2024-09-23 1900-01-01 07:50:11   
26159            1       1        1    2024-09-23 1

In [ ]:
tiempos_transaccion = {
    'No Identificada': 3.880916,
    'Tercera Edad': 5.760145,
    'Todo Publico': 4.791876
}

pivot_actualizado = agrupado.pivot_table(
    index=['Suc_Id', 'Dia_Semana', 'Hora','Factura_Fecha'],
    columns='Tipo_Plan',
    values='Cantidad_Transacciones',
    fill_value=0
).reset_index()

pivot_actualizado['Tiempo_Total'] = (
    pivot_actualizado['No Identificada'] * tiempos_transaccion['No Identificada'] +
    pivot_actualizado['Tercera Edad'] * tiempos_transaccion['Tercera Edad'] +
    pivot_actualizado['Todo Publico'] * tiempos_transaccion['Todo Publico']
)

pivot_actualizado['Personal_Necesario'] = np.ceil(pivot_actualizado['Tiempo_Total'] / 42)

pivot_actualizado['Dia_Semana'] = pivot_actualizado['Dia_Semana'].map({
    'Monday': 1,
    'Tuesday': 2,
    'Wednesday': 3,
    'Thursday': 4,
    'Friday': 5,
    'Saturday': 6,
    'Sunday': 7
})
pivot_actualizado['Transacciones_Totales'] = pivot_actualizado['Todo Publico'] + pivot_actualizado['Tercera Edad'] + pivot_actualizado['No Identificada']


pivot_actualizado = pivot_actualizado[['Suc_Id', 'Dia_Semana', 'Hora', 'Transacciones_Totales', 'Personal_Necesario']]



Pivot Actualizado:  Tipo_Plan
Suc_Id             361337
Dia_Semana         361337
Hora               361337
Factura_Fecha      361337
No Identificada    361337
Tercera Edad       361337
Todo Publico       361337
dtype: int64


In [30]:
pivot_actualizado.to_csv('personal_necesario.csv', index=False)